# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I Jérôme Pialat, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

In [54]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [73]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1963
df = spark.read.parquet(f'G:\\My Drive\\CourseWork\\processing-big-data-predict\\{year}\\')


In [74]:
df.show()

+----------+-------------------+-------------------+-------------------+-------------------+--------------------+-------+-----+
|      date|               open|               high|                low|              close|           adj_close| volume|stock|
+----------+-------------------+-------------------+-------------------+-------------------+--------------------+-------+-----+
|1963-10-28|  6.924644947052003|  6.940664768218994|   6.90862512588501|   6.90862512588501|  1.6844346523284912|  39900|   AA|
|1963-10-28|  6.488943099975586|  6.512368679046631|  6.465517044067383|  6.465517044067383|  1.5472991466522217|  42600| ARNC|
|1963-10-28| 0.5679012537002563| 0.5699588656425476| 0.5617284178733826| 0.5699588656425476| 0.11004561930894853| 789600|   BA|
|1963-10-28| 1.8541666269302368| 1.9166666269302368| 1.8541666269302368| 1.9166666269302368| 0.17308039963245392| 319200|  CAT|
|1963-10-28|                0.0|          4.0859375|          4.0234375|           4.046875| 0.067614793

## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [99]:
#TODO: Write your code here
Missingcheck = Check(spark, CheckLevel.Error, "Missing Values Check")

checkMissing = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        Missingcheck.isComplete('date') \
            .isComplete('open') \
            .isComplete('high') \
            .isComplete('low') \
            .isComplete('close') \
            .isComplete('adj_close') \
            .isComplete('volume') \
            .isComplete('stock')).run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkMissing)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+------------------+
|               check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+------------------+
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success|CompletenessConst...|          Success|                  |
|Missing Values Check|      Error|     Success

## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [76]:
#TODO: Write your code here
greaterThanZero = Check(spark, CheckLevel.Error, "Greater than zero check")

checkZeros = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        greaterThanZero.hasMin('open',lambda x:x > 0) \
                    .hasMin('high',lambda x:x > 0) \
                    .hasMin('low',lambda x:x > 0) \
                    .hasMin('close',lambda x:x > 0) \
                    .hasMin('adj_close',lambda x:x > 0) \
                    .hasMin('volume',lambda x:x > 0)).run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkZeros)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Failure|Value: 0.0 does n...|
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Success|                    |
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Success|                    |
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Success|                    |
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Success|                    |
|Greater than zero...|      Error|       Error|MinimumConstraint...|          Failure|Value: 0.0 does n...|
+--------------------+------

## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [81]:
#TODO: Write your code here
notNegative = Check(spark, CheckLevel.Error, "Not a negative check")

checkNegative = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        notNegative.isPositive('open') \
                    .isPositive('high') \
                    .isPositive('low') \
                    .isPositive('close') \
                    .isPositive('adj_close') \
                    .isPositive('volume')).run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkNegative)
checkResult_df.show()

+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|Not a negative check|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.45019920...|
|Not a negative check|      Error|       Error|ComplianceConstra...|          Success|                    |
|Not a negative check|      Error|       Error|ComplianceConstra...|          Success|                    |
|Not a negative check|      Error|       Error|ComplianceConstra...|          Success|                    |
|Not a negative check|      Error|       Error|ComplianceConstra...|          Success|                    |
|Not a negative check|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.99940239...|
+--------------------+------

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [96]:
#TODO: Write your code here
result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: stock: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 20,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 5020
    },
    "histogram": [
        [
            "NAV",
            251,
            0.05
        ],
        [
            "MO",
            251,
            0.05
        ],
        [
            "XOM",
            251,
            0.05
        ],
        [
            "HPQ",
            251,
            0.05
        ],
        [
            "AA",
            251,
            0.05
        ],
        [
            "PG",
            251,
            0.05
        ],
        [
            "GT",
            251,
            0.05
        ],
        [
            "FL",
            251,
            0.05
        ],
        [
            "IP",
            251,
            0.05
        ],
        [
       

## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [84]:
#TODO: Write your code here
metadata = spark.read.csv(f'G:\\My Drive\\CourseWork\\processing-big-data-predict\\Task2_data_profiling\\symbols_valid_meta.csv',header=True)
stockList = metadata.toPandas()['Symbol'].tolist()

validTickers = Check(spark, CheckLevel.Error, "Invalid Ticker")

checkTickers = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        validTickers.isContainedIn('stock',stockList)).run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkTickers)
checkResult_df.show()


+--------------+-----------+------------+--------------------+-----------------+------------------+
|         check|check_level|check_status|          constraint|constraint_status|constraint_message|
+--------------+-----------+------------+--------------------+-----------------+------------------+
|Invalid Ticker|      Error|     Success|ComplianceConstra...|          Success|                  |
+--------------+-----------+------------+--------------------+-----------------+------------------+



## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [82]:
#TODO: Write your code here
df = df.withColumn('UniqueId',F.concat('date','stock'))

noDuplicates = Check(spark, CheckLevel.Error, "Duplicate value")

checkDuplicates = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        noDuplicates.isUnique('UniqueId')).run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkDuplicates)
checkResult_df.show()
checkResult_df.collect()

+---------------+-----------+------------+--------------------+-----------------+------------------+
|          check|check_level|check_status|          constraint|constraint_status|constraint_message|
+---------------+-----------+------------+--------------------+-----------------+------------------+
|Duplicate value|      Error|     Success|UniquenessConstra...|          Success|                  |
+---------------+-----------+------------+--------------------+-----------------+------------------+



[Row(check='Duplicate value', check_level='Error', check_status='Success', constraint='UniquenessConstraint(Uniqueness(List(UniqueId),None))', constraint_status='Success', constraint_message='')]